# TOD_school_distribution_estimation

In [38]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna
from collections import OrderedDict
    
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import pandas as pd
import orca
# import os; os.chdir('/home/juan/activitysynth/')
import warnings; warnings.simplefilter('ignore')
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
from sklearn.neighbors import BallTree

import pylogit as pl                   # For choice model estimation
from pylogit import nested_logit as nl # For nested logit convenience funcs
import math 
from collections import OrderedDict 

import pickle
import dill
import time
import random
import scipy.stats as st

import urbansim_templates

from scripts import datasources, models, variables, utils

## Preparing data for simulation

In [7]:
input_file_format = 'csv'
input_data_dir = '/home/data/spring_2019/base/'

formattable_fname_dict = {
    'parcels': 'parcels.{0}',
    'buildings': 'buildings.{0}',
    'jobs': 'jobs.{0}',
    'establishments': 'establishments.{0}',
    'households': 'households.{0}',
    'persons': 'persons.{0}',
    'rentals': 'craigslist.{0}',
    'units': 'units.{0}',
    'mtc_skims': 'mtc_skims.{0}',
    'beam_skims_raw': '30.skims-smart-23April2019-baseline.csv.gz',
    'beam_skims_imputed': 'beam_skims_imputed.{0}',
    # the following nodes and edges .csv's will be phased out and
    # replaced by travel model skims entirely
    'drive_nodes': 'drive_nodes.{0}',
    'drive_edges': 'drive_edges.{0}',
    'drive_access_vars': 'drive_net_vars.{0}',
    'walk_nodes': 'walk_nodes.{0}',
    'walk_edges': 'walk_edges.{0}',
    'walk_access_vars': 'walk_net_vars.{0}',
    'zones': 'zones.{0}',
    'zone_access_vars': 'zones_w_access_vars.{0}',
}

def format_fname_dict(formattable_fname_dict, format='csv'):
    formatted_dict = {
        k: v.format('csv')
        for k, v in formattable_fname_dict.items()}
    return formatted_dict

input_fnames = format_fname_dict(
            formattable_fname_dict, input_file_format)

In [8]:
orca.add_injectable('input_file_format', input_file_format)
orca.add_injectable('input_data_dir', input_data_dir)
orca.add_injectable('input_fnames', input_fnames)
orca.add_injectable('store', None)

In [9]:
orca.run(['initialize_network_small', 'initialize_network_walk',
          'TOD_school_arrival_simulate','TOD_school_departure_simulate' ]) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Running step 'TOD_school_arrival_simulate'
Time to execute step 'TOD_school_arrival_simulate': 96.35 s
Running step 'TOD_school_departure_simulate'
Time to execute step 'TOD_school_departure_simulate': 8.82 s
Total time to execute iteration 1 with iteration value None: 105.17 s


## Simulation

In [158]:
@orca.step()
def TOD_school_distribution_simulate(persons):
    """
    Generate school arrival time of the day
    
    """
    #Running the simulation
    file_Name = "/home/juan/activitysynth/activitysynth/configs/TOD_school_distributions.pkl"
    fileObject = open(file_Name,'rb')  
    TOD_distributions = pickle.load(fileObject)

    #Extracting TOD category from persons 
    TOD_arrival = orca.get_table('persons').get_column('TOD_school_arrival')
    TOD_departure = orca.get_table('persons').get_column('TOD_school_departure')

    #Initializing series
    s_arrival = []
    s_departure = []
    
    for x in range(1,6):
        data_arrival = TOD_arrival[TOD_arrival == x]
        data_departure = TOD_departure[TOD_departure == x]
        
        name_arrival = 'TOD_arrival_'+ str(x)
        name_departure = 'TOD_departure_'+ str(x)

        #Getting distribution parameters
        dist_name_arrival, dist_params_arrival = TOD_distributions[name_arrival]
        eval_name_arrival = 'st.'+ dist_name_arrival + '.rvs'
        
        dist_name_departure, dist_params_departure = TOD_distributions[name_departure]
        eval_name_departure = 'st.'+ dist_name_departure + '.rvs'
            
        #Getting random draw from the distribution 
        random_draw_arrival = eval(eval_name_arrival)(*dist_params_arrival[:-2], 
                                                loc = dist_params_arrival[-2], 
                                                scale = dist_params_arrival[-1], 
                                                size = len(data_arrival))
        arrival = pd.Series(random_draw_arrival, index=data_arrival.index)
        
        random_draw_departure = eval(eval_name_departure)(*dist_params_departure[:-2], 
                                                loc = dist_params_departure[-2], 
                                                scale = dist_params_departure[-1], 
                                                size = len(data_departure))
        departure = pd.Series(random_draw_departure, index=data_departure.index)        
        
        s_arrival.append(arrival)
        s_departure.append(departure)
        
    s_arr = pd.concat(s_arrival)
    s_dep = pd.concat(s_departure)
    
    #Adding column to persons table
    persons.update_col('HS_ET', s_arr)
    persons.update_col('SH_ST', s_dep)

In [159]:
orca.run([ 'TOD_school_arrival_simulate','TOD_school_departure_simulate', 'TOD_school_distribution_simulate']) 

Running step 'TOD_school_arrival_simulate'
Time to execute step 'TOD_school_arrival_simulate': 9.06 s
Running step 'TOD_school_departure_simulate'
Time to execute step 'TOD_school_departure_simulate': 9.01 s
Running step 'TOD_school_distribution_simulate'
Time to execute step 'TOD_school_distribution_simulate': 1.45 s
Total time to execute iteration 1 with iteration value None: 19.52 s


In [160]:
persons = orca.get_table('persons').to_frame()

In [161]:
persons.loc[:,['HS_ET', 'SH_ST']][persons.TOD_school_arrival<=5]

,HS_ET,SH_ST
person_id,,
4754,7.589046,13.284286
4756,7.053550,14.537418
4760,6.876998,14.943477
4762,6.471967,14.242860
4766,7.428687,13.742664
4768,7.712350,14.913112
4770,6.973962,14.872455
4772,7.171744,13.649133
4774,5.609063,14.833145
